In [32]:
import tensorflow as tf
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:


import pandas as pd
import os
import numpy as np
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

import seaborn as sns
import matplotlib.pyplot as plt
import datatable as dt
import timer
js_path = '~/jane-street-market-prediction/'


In [7]:
SEED = 123

t = timer.Timer()
t.start()
import datatable as dt
data = dt.fread('~/jane-street-market-prediction/train.csv').to_pandas()
t.stop()

Elapsed time: 3.7233 seconds


In [8]:
import score_func

In [9]:
def neg_score(resp, weight, action):
    score = resp * weight * action 
    mask =  score < 0 
    
    return [np.mean(score[mask] ), np.std(score[mask] ), (sum(mask)/len(resp))]


In [10]:
score_func.u_score(data['resp'],data['weight'],1)

[-0.00019810473655664196, 0.1124218895745979]

In [11]:
score_func.pos_score(data['resp'],data['weight'],1)

[0.02652322642267673, 0.09900784345563303, 0.5892475646216614]

In [12]:
def pos_score(resp, weight, action):
    score = resp * weight * action 
    mask =  score >= 0 
    score[mask] 
    return [np.mean(score[mask] ), np.std(score[mask] )]

In [13]:
pos_score(data['resp'],data['weight'],1)

[0.02652322642267673, 0.09900784345563303]

In [14]:
profile = pd.pivot_table(data, index='date', values='resp',
           aggfunc={'resp':[np.sum, np.std, np.mean]},fill_value=0)
profile.sort_values(by='std')


,mean,std,sum
date,,,
294,0.000354,0.000703,0.010277
2,0.000766,0.003554,0.176875
147,0.000652,0.013513,2.275059
141,0.001496,0.013743,4.892174
419,-0.000408,0.014206,-1.573865
...,...,...,...
110,-0.007903,0.043108,-37.863545
19,0.008935,0.043416,54.084014
39,0.011139,0.046494,41.303268


In [15]:
vol_bins = pd.qcut(profile['std'], 4, labels=False)
data['vol_bin'] = 0
for vb in np.unique(vol_bins):
    mask = vol_bins == vb
    date_mask = data['date'].isin(vol_bins.loc[mask].index)
    data.loc[date_mask,'vol_bin'] = vb

In [16]:
data['resp_bin'] = pd.qcut(data['resp'], 10, labels=False)


In [17]:
data['resp_bin']


0          7
1          2
2          9
3          3
4          3
          ..
2390486    8
2390487    3
2390488    8
2390489    4
2390490    4
Name: resp_bin, Length: 2390491, dtype: int64

In [18]:
data.fillna(data.mean(),inplace=True)

data['short'] = ((data['resp'].values) < 0).astype(int)
data['action'] = ((data['resp'].values) >= 0).astype(int)


features = ['weight']+[c for c in data.columns if "feature" in c]
f_mean = np.mean(data[features[1:]].values,axis=0)

#features.append('weight')

In [19]:
features

['weight',
 'feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_19',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_24',
 'feature_25',
 'feature_26',
 'feature_27',
 'feature_28',
 'feature_29',
 'feature_30',
 'feature_31',
 'feature_32',
 'feature_33',
 'feature_34',
 'feature_35',
 'feature_36',
 'feature_37',
 'feature_38',
 'feature_39',
 'feature_40',
 'feature_41',
 'feature_42',
 'feature_43',
 'feature_44',
 'feature_45',
 'feature_46',
 'feature_47',
 'feature_48',
 'feature_49',
 'feature_50',
 'feature_51',
 'feature_52',
 'feature_53',
 'feature_54',
 'feature_55',
 'feature_56',
 'feature_57',
 'feature_58',
 'feature_59',
 'feature_60',
 'feature_61',
 'feature_62',
 'feature_63',
 'feature_64',
 'feature_65',
 'feature

In [20]:
###kfold by date
unique_dates = np.unique(data['date'] )
np.random.shuffle(unique_dates)
bags = 3
folds = 3
idx = np.linspace(start = 0, stop = len(unique_dates)-1, num = folds + 1, dtype = int)


In [21]:
###i = fold number 
i = 0
mask = data['date'].isin(unique_dates[idx[i]:idx[i+1]] )
features = ['weight']+[c for c in data.columns if "feature" in c]
all_features = ['weight']+[c for c in data.columns if "resp" in c]+ [c for c in data.columns if "feature" in c]
target = 'resp'

In [63]:
all_features

['weight',
 'feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18',
 'feature_19',
 'feature_20',
 'feature_21',
 'feature_22',
 'feature_23',
 'feature_24',
 'feature_25',
 'feature_26',
 'feature_27',
 'feature_28',
 'feature_29',
 'feature_30',
 'feature_31',
 'feature_32',
 'feature_33',
 'feature_34',
 'feature_35',
 'feature_36',
 'feature_37',
 'feature_38',
 'feature_39',
 'feature_40',
 'feature_41',
 'feature_42',
 'feature_43',
 'feature_44',
 'feature_45',
 'feature_46',
 'feature_47',
 'feature_48',
 'feature_49',
 'feature_50',
 'feature_51',
 'feature_52',
 'feature_53',
 'feature_54',
 'feature_55',
 'feature_56',
 'feature_57',
 'feature_58',
 'feature_59',
 'feature_60',
 'feature_61',
 'feature_62',
 'feature_63',
 'feature_64',
 'feature_65',
 'feature

In [21]:
i = 0
mask = data['date'].isin(unique_dates[idx[i]:idx[i+1]] )

train = data[~mask]
test = data[mask]

features = ['weight']+[c for c in data.columns if "feature" in c]
target = 'resp'
X_train = train.loc[:, features]
y_train = (train.loc[:, target])
X_test = test.loc[:, features]
y_test= (test.loc[:, target])

In [22]:
import ret_kfold
import imp
imp.reload(ret_kfold)

<module 'ret_kfold' from '/home/x99e/gitgood/legendary-robot/kaggle_jane_street/ret_kfold.py'>

In [23]:
mask = ret_kfold.ret_fold_mask(0)

In [28]:
x_train, y_train, x_test, y_test, test_weight, test_resp = ret_kfold.kfold_split(data,mask,features,target)
y_train_binary = (y_train > 0 ) * 1
y_test_binary = (y_test > 0 ) * 1

In [29]:
y_test_binary

0          1
1          0
2          1
3          0
4          0
          ..
2335939    0
2335940    0
2335941    1
2335942    0
2335943    1
Name: resp, Length: 469940, dtype: int64

In [31]:
def resp_MLP():
    hidden_units = 300
    model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(hidden_units, activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(hidden_units, activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(int(hidden_units/3), activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(1)
    ])    
    
    return model

In [42]:
js_path+'models/resp_model.h5'

'~/jane-street-market-prediction/models/resp_model.h5'

In [43]:
loss_fn = 'mse'
model = resp_MLP()
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
modelcheckpoint  = tf.keras.callbacks.ModelCheckpoint(js_path+'models/resp_model.h5', monitor='loss', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='adam',
              loss=loss_fn)
model.fit(x_train, y_train, batch_size= 10000, epochs=50 ,verbose  = 1, callbacks= [earlystopping])
model.evaluate(x_test, y_test)

Epoch 1/50
193/193 [==============================] - 3s 8ms/step - loss: 0.6578
Epoch 2/50
193/193 [==============================] - 2s 8ms/step - loss: 0.1240
Epoch 3/50
193/193 [==============================] - 2s 8ms/step - loss: 0.0324
Epoch 4/50
193/193 [==============================] - 2s 8ms/step - loss: 0.0105
Epoch 5/50
193/193 [==============================] - 2s 8ms/step - loss: 0.0032
Epoch 6/50
193/193 [==============================] - 2s 9ms/step - loss: 0.0012
Epoch 7/50
193/193 [==============================] - 2s 9ms/step - loss: 8.3761e-04
Epoch 8/50
193/193 [==============================] - 2s 9ms/step - loss: 7.5424e-04
Epoch 9/50
193/193 [==============================] - 2s 9ms/step - loss: 7.4184e-04
Epoch 10/50
193/193 [==============================] - 2s 9ms/step - loss: 7.3998e-04
Epoch 11/50
193/193 [==============================] - 2s 9ms/step - loss: 7.4244e-04
Epoch 12/50
193/193 [==============================] - 2s 8ms/step - loss: 7.4261e-04
E

0.0006559767061844468

In [79]:
model.evaluate(x_test, y_test)

14686/14686 [==============================] - 25s 2ms/step - loss: 6.5598e-04


0.0006559767061844468

In [78]:
preds = model.predict(x_test)

pred_action = (preds > 0) * 1
score_func.u(test_weight, test_resp,np.array(pred_action).reshape(len(y_test),))

[194.42339828374926]

In [86]:
def binary_MLP():
    hidden_units = 300
    model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(hidden_units, activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(hidden_units, activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(int(hidden_units/3), activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(1)
    ])    
    
    return model

In [27]:
y_train_binary = (y_train > 0 ) * 1
y_test_binary = (y_test > 0 ) * 1

In [88]:
y_train_binary

5587       0
5588       0
5589       1
5590       0
5591       1
          ..
2390486    1
2390487    0
2390488    1
2390489    0
2390490    0
Name: resp, Length: 1920551, dtype: int64

In [90]:
loss_fn = tf.keras.losses.BinaryCrossentropy()
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)


model = binary_MLP()
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['AUC'])
model.fit(x_train, y_train_binary, batch_size= 10000, epochs=50 ,verbose  = 1 ,callbacks= [earlystopping])
model.evaluate(x_test, y_test_binary)

Epoch 1/50
193/193 [==============================] - 3s 9ms/step - loss: 5.7276 - accuracy: 0.5022
Epoch 2/50
193/193 [==============================] - 2s 9ms/step - loss: 5.6894 - accuracy: 0.5003
Epoch 3/50
193/193 [==============================] - 2s 9ms/step - loss: 4.6014 - accuracy: 0.4986
Epoch 4/50
193/193 [==============================] - 2s 9ms/step - loss: 4.2170 - accuracy: 0.5029
Epoch 5/50
193/193 [==============================] - 2s 9ms/step - loss: 4.4137 - accuracy: 0.5009
Epoch 6/50
193/193 [==============================] - 2s 9ms/step - loss: 2.3037 - accuracy: 0.5010
Epoch 7/50
193/193 [==============================] - 2s 9ms/step - loss: 1.7840 - accuracy: 0.5008
Epoch 8/50
193/193 [==============================] - 2s 9ms/step - loss: 1.3438 - accuracy: 0.5000
Epoch 9/50
193/193 [==============================] - 2s 9ms/step - loss: 1.3341 - accuracy: 0.5002
Epoch 10/50
193/193 [==============================] - 2s 9ms/step - loss: 1.1989 - accuracy: 0.5001

[0.8800592422485352, 0.0]

In [93]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])
pred_train = probability_model(x_train.values)
pred_test = probability_model(x_test.values)


In [ ]:

pred_bins = [] 
for n_pred in pred_test:
    pred_bins.append(np.argmax(n_pred))


In [1]:
pred_test.numpy()

NameError: name 'pred_test' is not defined

In [31]:


def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model


batch_size = 5000
hidden_units = [150, 150, 150]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

clf = create_mlp(
    len(features), 1, hidden_units, dropout_rates, label_smoothing, learning_rate
    )

clf.fit(x_train, y_train_binary, epochs=200, batch_size=5000)

Epoch 1/200
385/385 [==============================] - 5s 7ms/step - loss: 0.7071 - AUC: 0.5112
Epoch 2/200
385/385 [==============================] - 3s 7ms/step - loss: 0.6922 - AUC: 0.5246
Epoch 3/200
385/385 [==============================] - 3s 8ms/step - loss: 0.6914 - AUC: 0.5302
Epoch 4/200
385/385 [==============================] - 3s 8ms/step - loss: 0.6911 - AUC: 0.5324
Epoch 5/200
385/385 [==============================] - 3s 8ms/step - loss: 0.6907 - AUC: 0.5347
Epoch 6/200
385/385 [==============================] - 3s 7ms/step - loss: 0.6906 - AUC: 0.5357
Epoch 7/200
385/385 [==============================] - 3s 8ms/step - loss: 0.6903 - AUC: 0.5372
Epoch 8/200
385/385 [==============================] - 3s 8ms/step - loss: 0.6900 - AUC: 0.5387
Epoch 9/200
385/385 [==============================] - 3s 8ms/step - loss: 0.6898 - AUC: 0.5395
Epoch 10/200
385/385 [==============================] - 3s 8ms/step - loss: 0.6895 - AUC: 0.5409
Epoch 11/200
385/385 [=================

385/385 [==============================] - 3s 8ms/step - loss: 0.6785 - AUC: 0.5718
Epoch 169/200
385/385 [==============================] - 3s 8ms/step - loss: 0.6783 - AUC: 0.5720
Epoch 170/200
385/385 [==============================] - 3s 8ms/step - loss: 0.6782 - AUC: 0.5722
Epoch 171/200
385/385 [==============================] - 3s 7ms/step - loss: 0.6785 - AUC: 0.5714
Epoch 172/200
385/385 [==============================] - 3s 8ms/step - loss: 0.6782 - AUC: 0.5729
Epoch 173/200
385/385 [==============================] - 3s 7ms/step - loss: 0.6785 - AUC: 0.5720
Epoch 174/200
385/385 [==============================] - 3s 7ms/step - loss: 0.6785 - AUC: 0.5715
Epoch 175/200
385/385 [==============================] - 3s 7ms/step - loss: 0.6783 - AUC: 0.5721
Epoch 176/200
385/385 [==============================] - 3s 8ms/step - loss: 0.6784 - AUC: 0.5718
Epoch 177/200
385/385 [==============================] - 3s 7ms/step - loss: 0.6783 - AUC: 0.5720
Epoch 178/200
385/385 [===========

In [45]:
preds.shape

(469940, 1)

In [46]:
np.array(y_test).reshape(len(y_test),).shape

(469940,)

In [56]:
x_train['weight']

5587       9.821427
5588       0.838150
5589       0.115654
5590       0.000000
5591       0.000000
             ...   
2390486    0.000000
2390487    0.000000
2390488    0.000000
2390489    0.283405
2390490    0.000000
Name: weight, Length: 1920551, dtype: float64

In [58]:
test_w = data[mask].loc[:,'weight']
test_resp = data[mask].loc[:,'resp']
test_action = (test_resp > 0) * 1
test_size= len(test_resp)

In [60]:
x_test

,weight,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129
0,0.000000,1,-1.872746,-2.191242,-0.474163,-0.323046,0.014688,-0.002484,0.051777,0.026828,...,0.335127,0.268776,1.168391,8.313583,1.782433,14.018213,2.653056,12.600292,2.301488,11.445807
1,16.673515,-1,-1.349537,-1.704709,0.068058,0.028432,0.193794,0.138212,0.051777,0.026828,...,0.335127,0.268776,-1.178850,1.777472,-0.915458,2.831612,-1.417010,2.297459,-1.304614,1.898684
2,0.000000,-1,0.812780,-0.256156,0.806463,0.400221,-0.614188,-0.354800,0.051777,0.026828,...,0.335127,0.268776,6.115747,9.667908,5.542871,11.671595,7.281757,10.060014,6.638248,9.427299
3,0.000000,-1,1.174378,0.344640,0.066872,0.009357,-1.006373,-0.676458,0.051777,0.026828,...,0.335127,0.268776,2.838853,0.499251,3.033732,1.513488,4.397532,1.266037,3.856384,1.013469
4,0.138531,1,-3.172026,-3.093182,-0.161518,-0.128149,-0.195006,-0.143780,0.051777,0.026828,...,0.335127,0.268776,0.344850,4.101145,0.614252,6.623456,0.800129,5.233243,0.362636,3.926633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335939,0.000000,1,-1.953012,-1.825455,1.167282,1.407730,0.604182,0.745740,0.533140,0.860828,...,-1.821456,-2.384486,-1.125181,0.374372,-1.678955,-2.763350,-2.406755,-1.314956,-1.421907,0.109313
2335940,0.000000,-1,-1.093580,-0.308872,-1.175020,-1.802869,-0.266547,-0.411329,-0.622232,-0.924464,...,-3.044894,-0.937643,-1.864301,2.394599,-2.370054,-0.312073,-3.317327,1.023902,-2.057109,2.593925
2335941,0.000000,-1,-1.140352,-0.681436,-0.155998,-0.173413,1.272839,1.825249,0.102961,0.315304,...,-1.472322,-2.130481,-1.143322,-0.328897,-1.453351,-2.812128,-2.248055,-1.705126,-1.632931,-0.999637
2335942,0.000000,-1,1.911317,0.753140,-1.752795,-1.096010,1.310088,0.788032,-0.760948,-0.698112,...,1.673762,-1.070076,2.878417,0.354457,1.798903,-0.709118,3.122721,-0.039724,3.135445,0.229055


In [59]:

pred_action = (preds > 0) * 1
score_func.u(x_test['weight'],test_resp,np.array(pred_action).reshape(len(y_test),)) 



[194.42339828374926]

In [72]:
score_func.u(test_weight, test_resp,np.array(pred_action).reshape(len(y_test),))

[194.42339828374926]

In [49]:
def pred_threshold(thd):
    pred_action = (preds > thd) * 1
    score =  score_func.u(test_w,test_resp,np.array(pred_action).reshape(len(test_w),)) 
    return np.array(score).astype('double') * -1



In [50]:
from scipy import optimize
sresult = optimize.minimize_scalar(pred_threshold)

In [51]:
sresult.x

array([0.00090106])

In [52]:
import imp
imp.reload(score_func)

<module 'score_func' from '/home/x99e/gitgood/legendary-robot/kaggle_jane_street/score_func.py'>

In [53]:
test_action.shape

(469940,)

In [54]:
 np.array(test_resp) * np.array(test_w) * np.array(pred_action).reshape(len(y_test),)

array([ 0., -0.,  0., ...,  0., -0.,  0.])

In [63]:
score_func.u(test_w,test_resp,test_action) 

[12979.074287617968]

In [64]:
score_func.u(test_w,test_resp,np.array(pred_action).reshape(len(y_test),)) 

[309.93456452215366]

In [65]:
score_func.pos_score(test_w,test_resp,np.array(pred_action).reshape(len(y_test),)) 

[0.008834356337043228, 0.042206384155710534, 0.7557011468372646]

In [94]:
score_func.u(test_w,test_resp,np.random.randint(0,2,size=len(test_resp))) 

[-99.07358774971269]

In [79]:
score_func.neg_score(test_w,test_resp,np.random.randint(0,2,size=len(test_resp))) 

[-0.04021211768006526, 0.12885324404099371, 0.2043161805375402]

In [81]:
score_func.pos_score(test_w,test_resp,np.random.randint(0,2,size=len(test_resp))) 

[0.010026132783883944, 0.06097105191575509, 0.7955002667632561]

In [71]:
np.random.randint(0,2,size=len(test_resp))

array([1, 1, 1, ..., 1, 1, 0])

In [13]:
def seq_MLP2():
    hidden_units = 300
    model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(hidden_units, activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(hidden_units, activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(int(hidden_units/3), activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
    ])    
    
    return model

In [14]:
def seq_MLP():
    hidden_units = 300
    model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(hidden_units, activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(hidden_units, activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(int(hidden_units/3), activation='relu'),
      tf.keras.layers.BatchNormalization(),  
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(1)
    ])    
    
    return model

In [29]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model = seq_MLP()
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size= 10000, epochs=50 ,verbose  = 1 )
model.evaluate(X_test, y_test)

Epoch 1/50
193/193 [==============================] - 3s 9ms/step - loss: 2.3094 - accuracy: 0.1647
Epoch 2/50
193/193 [==============================] - 2s 9ms/step - loss: 2.1406 - accuracy: 0.1870
Epoch 3/50
193/193 [==============================] - 2s 9ms/step - loss: 2.1252 - accuracy: 0.1897
Epoch 4/50
193/193 [==============================] - 2s 9ms/step - loss: 2.1177 - accuracy: 0.1923
Epoch 5/50
193/193 [==============================] - 2s 9ms/step - loss: 2.1123 - accuracy: 0.1937
Epoch 6/50
193/193 [==============================] - 2s 9ms/step - loss: 2.1088 - accuracy: 0.1944
Epoch 7/50
193/193 [==============================] - 2s 9ms/step - loss: 2.1060 - accuracy: 0.1955
Epoch 8/50
193/193 [==============================] - 2s 9ms/step - loss: 2.1021 - accuracy: 0.1965
Epoch 9/50
193/193 [==============================] - 2s 9ms/step - loss: 2.1001 - accuracy: 0.1974
Epoch 10/50
193/193 [==============================] - 2s 9ms/step - loss: 2.0973 - accuracy: 0.1983

[2.149144172668457, 0.18434061110019684]

In [16]:
model = seq_MLP()
model.compile(optimizer='adam',
              loss='mse')
model.fit(X_train, y_train, batch_size= 10000, epochs=50 ,verbose  = 1 )
model.evaluate(X_test, y_test)

Epoch 1/50
189/189 [==============================] - 3s 8ms/step - loss: 23.4657
Epoch 2/50
189/189 [==============================] - 2s 8ms/step - loss: 9.4292
Epoch 3/50
189/189 [==============================] - 2s 8ms/step - loss: 8.9414
Epoch 4/50
189/189 [==============================] - 2s 8ms/step - loss: 8.6872
Epoch 5/50
189/189 [==============================] - 2s 8ms/step - loss: 8.5498
Epoch 6/50
189/189 [==============================] - 2s 8ms/step - loss: 8.4191
Epoch 7/50
189/189 [==============================] - 2s 8ms/step - loss: 8.3277
Epoch 8/50
189/189 [==============================] - 2s 8ms/step - loss: 8.2679
Epoch 9/50
189/189 [==============================] - 2s 8ms/step - loss: 8.2077
Epoch 10/50
189/189 [==============================] - 2s 8ms/step - loss: 8.1614
Epoch 11/50
189/189 [==============================] - 2s 8ms/step - loss: 8.1431
Epoch 12/50
189/189 [==============================] - 2s 8ms/step - loss: 8.1040
Epoch 13/50
189/189 [===

8.88668441772461

In [26]:
model.evaluate(X_test, y_test)

14620/14620 [==============================] - 31s 2ms/step - loss: 1.5392 - accuracy: 0.2910


[1.5392255783081055, 0.2909947633743286]

In [39]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])
pred_train = probability_model(X_train.values)
pred_test = probability_model(X_test.values)

pred_bins = [] 
for n_pred in pred_test:
    pred_bins.append(np.argmax(n_pred))


In [40]:
skl_pred_score(pred_bins,y_test)

Accuracy: 0.295488
Precision: 0.323100
Recall: 0.297252
F1 score: 0.292774


In [133]:

import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param = {
    'max_depth': 10,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 3}  # the number of classes that exist in this datset
num_round = 20  # the number of training iterations
bst = xgb.train(param, dtrain, num_round)
preds = bst.predict(dtest)

best_preds = np.asarray([np.argmax(line) for line in preds])
#print "Numpy array precision:", precision_score(y_test, best_preds, average='macro')



[23:40:24] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:40:52] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [145]:
np.mean(np.array(best_preds)==y_test)



0.45103186727225814

In [146]:
np.mean((np.array(best_preds)-y_test)**2)


0.9448906290315314

In [143]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])
predictions = probability_model(X_train.values)


In [148]:
pred_bins = [] 
for preds in predictions:
    pred_bins.append(np.argmax(preds))


In [154]:
stats = (pred_bins - data['vol_bin']) ** 2

In [156]:
stats.mean() ** 0.5

2.8112875042434053

In [27]:
#gbm = xgb.XGBClassifier(max_depth=5, n_estimators=300, learning_rate=0.05).fit(X_train, y_train)
#gbm_pred = gbm.predict(X_test)

In [106]:
hidden_units = [300,130]
dropout_rates = [0.2,0.2,0.2]

num_labels = len(np.unique(y_train))
num_labels
i = 0
inp = tf.keras.layers.Input(shape=(X_train.shape[1]))
x = tf.keras.layers.BatchNormalization()(inp)
x = tf.keras.layers.Dropout(0.2)(x) 


x = tf.keras.layers.Dense(hidden_units[i])(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
x = tf.keras.layers.Dropout(dropout_rates[i])(x)

x = tf.keras.layers.Dense(num_labels)(x)


out = tf.keras.layers.Activation("softmax")(x)

k_model = tf.keras.models.Model(inputs=inp, outputs=out)



In [108]:
k_model.summary()

k_model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
k_model.fit(X_train, y_train, batch_size= 10000, epochs=50)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 131)]             0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 131)               524       
_________________________________________________________________
dropout_22 (Dropout)         (None, 131)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 300)               39600     
_________________________________________________________________
batch_normalization_23 (Batc (None, 300)               1200      
_________________________________________________________________
activation (Activation)      (None, 300)               0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 300)               0     

In [109]:
def vol_bin_classifier(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns))
    x = tf.keras.layers.BatchNormalization()(inp)
    x= tf.keras.layers.Dropout(dropout_rates[0])(x) 
    #x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dense(hidden_units[0])(x)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x) #(x)
    
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)
        x = tf.keras.layers.Concatenate(axis=1)([x, inp])
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    
    out = tf.keras.layers.Activation("softmax")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        #optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])

    return model

In [110]:
def vb_simple(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns))
    
    for _ in range(3):
        x = tf.keras.layers.Dense(hidden_units[0], activation='relu')(inp)
        #x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout_rates[0])(x)



    x= tf.keras.layers.Dense(num_labels)(x)
    
    out = tf.keras.layers.Activation("softmax")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        #optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])

    return model

In [112]:
inp_shape = X_train.shape[1]
num_cats = len(np.unique(y_train))
batch_size = 5000
hidden_units = [300, 300, 300]
dropout_rates = [0.2, 0.2, 0.2, 0.2]
label_smoothing = 1e-2
learning_rate = 1e-3

In [116]:
fmodel = vb_simple(
    inp_shape,num_cats, hidden_units, dropout_rates, label_smoothing, learning_rate
    )
fmodel.fit(X_train, y_train, epochs=100, batch_size=20000)
fmodel.evaluate(X_test, y_test)

Epoch 1/100
96/96 [==============================] - 1s 10ms/step - loss: 2.5179 - accuracy: 0.1255
Epoch 2/100
96/96 [==============================] - 1s 10ms/step - loss: 2.2758 - accuracy: 0.1520
Epoch 3/100
96/96 [==============================] - 1s 10ms/step - loss: 2.2349 - accuracy: 0.1631
Epoch 4/100
96/96 [==============================] - 1s 10ms/step - loss: 2.2173 - accuracy: 0.1696
Epoch 5/100
96/96 [==============================] - 1s 10ms/step - loss: 2.2069 - accuracy: 0.1742
Epoch 6/100
96/96 [==============================] - 1s 10ms/step - loss: 2.1981 - accuracy: 0.1784
Epoch 7/100
96/96 [==============================] - 1s 10ms/step - loss: 2.1913 - accuracy: 0.1819
Epoch 8/100
96/96 [==============================] - 1s 10ms/step - loss: 2.1857 - accuracy: 0.1851
Epoch 9/100
96/96 [==============================] - 1s 10ms/step - loss: 2.1800 - accuracy: 0.1879
Epoch 10/100
96/96 [==============================] - 1s 10ms/step - loss: 2.1747 - accuracy: 0.1907

In [117]:
fmodel.evaluate(X_test, y_test)

15261/15261 [==============================] - 27s 2ms/step - loss: 2.3678 - accuracy: 0.1339


[2.367835283279419, 0.13387653231620789]

In [118]:



fmodel = vol_bin_classifier(
    inp_shape,num_cats, hidden_units, dropout_rates, label_smoothing, learning_rate
    )
fmodel.fit(X_train, y_train, epochs=100, batch_size=20000)
fmodel.evaluate(X_test, y_test)
#fmodel.summary()


Epoch 1/100
96/96 [==============================] - 5s 30ms/step - loss: 2.5185 - accuracy: 0.1269
Epoch 2/100
96/96 [==============================] - 3s 31ms/step - loss: 2.2631 - accuracy: 0.1497
Epoch 3/100
96/96 [==============================] - 3s 30ms/step - loss: 2.2391 - accuracy: 0.1569
Epoch 4/100
96/96 [==============================] - 3s 31ms/step - loss: 2.2205 - accuracy: 0.1645
Epoch 5/100
96/96 [==============================] - 3s 31ms/step - loss: 2.2012 - accuracy: 0.1742
Epoch 6/100
96/96 [==============================] - 3s 31ms/step - loss: 2.1814 - accuracy: 0.1838
Epoch 7/100
96/96 [==============================] - 3s 31ms/step - loss: 2.1649 - accuracy: 0.1916
Epoch 8/100
96/96 [==============================] - 3s 30ms/step - loss: 2.1492 - accuracy: 0.1990
Epoch 9/100
96/96 [==============================] - 3s 29ms/step - loss: 2.1348 - accuracy: 0.2050
Epoch 10/100
96/96 [==============================] - 3s 29ms/step - loss: 2.1222 - accuracy: 0.2109

[2.5122361183166504, 0.1327277421951294]

In [123]:
fmodel.fit(X_train, y_train, epochs=20, batch_size=20000)

Epoch 1/200
120/120 [==============================] - 3s 26ms/step - loss: 2.1491 - accuracy: 0.1953
Epoch 2/200
120/120 [==============================] - 3s 26ms/step - loss: 2.1471 - accuracy: 0.1965
Epoch 3/200
120/120 [==============================] - 3s 26ms/step - loss: 2.1447 - accuracy: 0.1977
Epoch 4/200
120/120 [==============================] - 3s 26ms/step - loss: 2.1430 - accuracy: 0.1985
Epoch 5/200
120/120 [==============================] - 3s 26ms/step - loss: 2.1413 - accuracy: 0.1993
Epoch 6/200
120/120 [==============================] - 3s 25ms/step - loss: 2.1392 - accuracy: 0.2001
Epoch 7/200
120/120 [==============================] - 3s 26ms/step - loss: 2.1381 - accuracy: 0.2007
Epoch 8/200
120/120 [==============================] - 3s 25ms/step - loss: 2.1361 - accuracy: 0.2017
Epoch 9/200
120/120 [==============================] - 3s 26ms/step - loss: 2.1347 - accuracy: 0.2025
Epoch 10/200
120/120 [==============================] - 3s 25ms/step - loss: 2.133

120/120 [==============================] - 3s 26ms/step - loss: 2.0821 - accuracy: 0.2257
Epoch 160/200
120/120 [==============================] - 3s 25ms/step - loss: 2.0820 - accuracy: 0.2262
Epoch 161/200
120/120 [==============================] - 3s 25ms/step - loss: 2.0823 - accuracy: 0.2260
Epoch 162/200
120/120 [==============================] - 3s 25ms/step - loss: 2.0818 - accuracy: 0.2262
Epoch 163/200
120/120 [==============================] - 3s 25ms/step - loss: 2.0819 - accuracy: 0.2260
Epoch 164/200
120/120 [==============================] - 3s 25ms/step - loss: 2.0818 - accuracy: 0.2261
Epoch 165/200
120/120 [==============================] - 3s 25ms/step - loss: 2.0817 - accuracy: 0.2262
Epoch 166/200
120/120 [==============================] - 3s 25ms/step - loss: 2.0812 - accuracy: 0.2265
Epoch 167/200
120/120 [==============================] - 3s 25ms/step - loss: 2.0811 - accuracy: 0.2267
Epoch 168/200
120/120 [==============================] - 3s 25ms/step - loss: 

In [13]:
import gc
from time import time
import multiprocessing

import numpy as np
import pandas as pd
import xgboost as xgb

In [28]:
multiprocessing.cpu_count()

24

In [12]:
import timer
t = timer.Timer()
t.start()

t.stop()

Elapsed time: 0.0000 seconds


In [56]:
import gc
gc.collect()
t.stop()

TimerError: Timer is not running. Use .start() to start it

In [18]:
# Settings
NAN_VALUE = -9999
#features = [c for c in data.columns if 'feature' in c]
#target = 'vol_bin'

# Split into features X and target Y
#X = data.loc[:, features].fillna(NAN_VALUE)
#Y = (data.loc[:, target] > 0).astype(int)

# Clear memory
#del data
#gc.collect()

# Train model
t.start()
model = xgb.XGBClassifier(
    n_estimators = 500,
    tree_method='gpu_hist',
    nthread=multiprocessing.cpu_count()-1
)
model.fit(X_train, y_train)
print('Finished training model')

t.stop()

[05:02:42] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Finished training model
Elapsed time: 100.4053 seconds


In [17]:
model.get_params()

{'objective': 'multi:softprob',
 'use_label_encoder': True,
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'gpu_id': 0,
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 100,
 'n_jobs': 23,
 'num_parallel_tree': 1,
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': None,
 'subsample': 1,
 'tree_method': 'gpu_hist',
 'validate_parameters': 1,
 'verbosity': None,
 'nthread': 23}

In [22]:

y_pred = model.predict(X_test)

In [16]:
y_test

22440      1
22441      1
22442      1
22443      1
22444      1
          ..
2390486    2
2390487    2
2390488    2
2390489    2
2390490    2
Name: vol_bin, Length: 479681, dtype: int64

In [21]:
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

 

In [25]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_pred, y_test)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_pred, y_test, average = 'macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_pred, y_test,average = 'macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_pred, y_test,average = 'macro')
print('F1 score: %f' % f1)

Accuracy: 0.318647
Precision: 0.327637
Recall: 0.312099
F1 score: 0.312950


In [26]:
def skl_pred_score(y_pred, y_test):
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_pred, y_test)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_pred, y_test, average = 'macro')
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_pred, y_test,average = 'macro')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_pred, y_test,average = 'macro')
    print('F1 score: %f' % f1)

In [141]:
model.predict(X_train[0:1])


array([[-3.1406872 , -5.133367  ,  0.33794835, -1.5648116 , -1.4330602 ,
        -1.9282924 ,  0.29217353,  1.996077  ,  1.6452407 ,  1.560955  ]],
      dtype=float32)

In [142]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])
predictions = probability_model(X_train.values)


In [143]:
predictions = probability_model(X_train[0:2].values)


In [144]:
predictions[0]

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([2.0905882e-03, 2.8500901e-04, 6.7767411e-02, 1.0107967e-02,
       1.1531389e-02, 7.0275711e-03, 6.4735338e-02, 3.5574403e-01,
       2.5047842e-01, 2.3023234e-01], dtype=float32)>

In [145]:
np.argmax(predictions[0])


7

In [77]:
data['vol_bin']

0          7
1          7
2          7
3          7
4          7
          ..
2390486    6
2390487    6
2390488    6
2390489    6
2390490    6
Name: vol_bin, Length: 2390491, dtype: int64

In [ ]:
from math import sqrt

def utility_score(df,action_vec):
    """Calculate utility score of a dataframe according to formulas defined at
    https://www.kaggle.com/c/jane-street-market-prediction/overview/evaluation
    """

    p = df['weight']  * df['resp'] * action_vec
    df['p'] = p
    p_i = df.set_index('date')['p'].groupby('date').sum()
    t = (p_i.sum() / sqrt((p_i**2).sum())) * (sqrt(250 / p_i.index.size))
    return min(max(t, 0), 6) * p_i.sum()

In [ ]:
pred_df = train[['date', 'resp','weight']]
pred_df['actual'] = ((train['resp'].values) > 0).astype(int)
preds = clf(X_train.values, training=False)
pred_df['preds'] = preds.numpy()
pred_df['action'] = (pred_df['preds'] >= 0.5) * 1

In [ ]:
utility_score(pred_df, pred_df['action']) / utility_score(pred_df, pred_df['actual'])


In [36]:
X_train[:1].shape

(1, 131)

In [58]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

predictions = model(x_train[:1]).numpy()

predictions

array([[-0.0271607 , -0.78438675,  0.7532629 ,  0.9322199 ,  0.18562594,
         0.58452153,  0.14282605,  0.5791917 ,  0.18594626, -0.60336185]],
      dtype=float32)

In [59]:
tf.nn.softmax(predictions).numpy()

array([[0.07061819, 0.0331175 , 0.15411688, 0.18431908, 0.08736322,
        0.13018675, 0.08370297, 0.12949471, 0.08739121, 0.0396895 ]],
      dtype=float32)

In [60]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()

2.0387852

In [20]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [21]:
model.fit(x_train, y_train, batch_size= 1000, epochs=20)

Epoch 1/20
60/60 [==============================] - 0s 2ms/step - loss: 1.3377 - accuracy: 0.6045
Epoch 2/20
60/60 [==============================] - 0s 2ms/step - loss: 0.3775 - accuracy: 0.8934
Epoch 3/20
60/60 [==============================] - 0s 2ms/step - loss: 0.2971 - accuracy: 0.9158
Epoch 4/20
60/60 [==============================] - 0s 2ms/step - loss: 0.2481 - accuracy: 0.9301
Epoch 5/20
60/60 [==============================] - 0s 3ms/step - loss: 0.2113 - accuracy: 0.9397
Epoch 6/20
60/60 [==============================] - 0s 3ms/step - loss: 0.1895 - accuracy: 0.9448
Epoch 7/20
60/60 [==============================] - 0s 2ms/step - loss: 0.1760 - accuracy: 0.9504
Epoch 8/20
60/60 [==============================] - 0s 3ms/step - loss: 0.1548 - accuracy: 0.9561
Epoch 9/20
60/60 [==============================] - 0s 2ms/step - loss: 0.1450 - accuracy: 0.9590
Epoch 10/20
60/60 [==============================] - 0s 2ms/step - loss: 0.1346 - accuracy: 0.9616
Epoch 11/20
60/60 [

In [22]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0815 - accuracy: 0.9747


[0.08148473501205444, 0.9746999740600586]

In [24]:
model.predict(x_test[0:1])

array([[ -4.3066444 ,  -7.0519342 ,   0.6971853 ,   3.110398  ,
         -9.046184  ,  -2.5873153 , -10.296194  ,  10.261124  ,
         -2.562631  ,  -0.04820664]], dtype=float32)

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])
predictions = probability_model(x_test[:1])
np.argmax(predictions[0])

In [27]:
predictions = probability_model(x_test[:1])

In [28]:
np.argmax(predictions[0])


7

model.summary

In [53]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [25]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [54]:
x_train.shape

(60000, 28, 28)

In [55]:
X_train.shape

(2390491, 131)

In [ ]:
def make_resLSTM(i_shape, multiplier = 1):
    
    inputs =  Input(shape = i_shape)
    n_timesteps, n_features = i_shape[0], i_shape[1]

    node_factor = int(n_features * multiplier)

    ######### resnet LSTM model 
    stackLSTM0 = LSTM(node_factor * 10, activation='relu', input_shape = i_shape, return_sequences=True)(inputs)
    stackLSTM0 = BatchNormalization()(stackLSTM0)
    merge0 = concatenate([stackLSTM0, inputs])
    
    
    stackLSTM1 = LSTM(node_factor * 5, activation='relu', return_sequences=True)(merge0)
    stackLSTM1 = BatchNormalization()(stackLSTM1)
    merge1 = concatenate([stackLSTM1, inputs])
    
        
    stackLSTM2 = LSTM(node_factor * 2, activation='relu', return_sequences=True)(merge1)
    stackLSTM2 = BatchNormalization()(stackLSTM2)
    merge2 = concatenate([stackLSTM2, inputs])
    
    
    resLSTM = LSTM(node_factor * 10, activation='relu', return_sequences=False)(merge2)
    resLSTM = BatchNormalization()(resLSTM)
    resLSTM = Dropout(rate = 0.3)(resLSTM)
    # resLSTM = Flatten()(resLSTM)
    resLSTM = Dense(node_factor * 10, activation='relu')(resLSTM)
    resLSTM = Dropout(rate = 0.3)(resLSTM)
    resLSTM = BatchNormalization()(resLSTM)
    
    

    # output
    output = Dense(node_factor * 1, activation='relu',kernel_regularizer='l1')(resLSTM)
    # output = BatchNormalization()(output)
    output = Dense(node_factor * 1, activation='relu')(output)
    output = Dense(1)(output)
    model = Model(inputs=inputs, outputs=output, name = 'resnet')
    # summarize layers
    print(model.summary())


    return model 